### Please download and install the following packages before proceeding:

#### 1. pip install uszipcode
#### 2. pip install folium
#### 3. pip install memory_profiler

### Please dowload and unzip the following files and save in your working directory:

#### 1. crime.csv
#### 2. crime_classification
#### 3. zip_attributes

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import string
import random
from scipy import stats
import statsmodels.formula.api as sm
import os
import folium

from sklearn.cluster import KMeans
from folium import plugins
%load_ext memory_profiler
%matplotlib inline

## Read in the main data file
The encoding for the file is different and so we need to specify that while reading it in

In [2]:
# THIS CODE CELL IS TO CAPTURE THE START TIME OF OUR NOTEBOOK RUN
start_time = time.time()

In [3]:
crime = pd.read_csv('crime.csv', encoding = 'ISO-8859-1')

In [4]:
#crime = crime[crime.SHOOTING !=NaN]
crime = crime.dropna(how='any', subset=['SHOOTING'])

In [5]:
crime.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
759,I182053533,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,909,Y,2018-07-07 23:33:00,2018,7,Saturday,23,Part One,WHITTIER ST,42.333802,-71.088809,"(42.33380238, -71.08880867)"
869,I182053414,3001,Medical Assistance,DEATH INVESTIGATION,B2,316,Y,2018-07-07 15:06:00,2018,7,Saturday,15,Part Three,HOMESTEAD ST,42.312421,-71.091549,"(42.31242132, -71.09154902)"
1602,I182052602,3001,Medical Assistance,DEATH INVESTIGATION,B2,324,Y,2018-07-04 21:50:00,2018,7,Wednesday,21,Part Three,BROOKFORD ST,42.318415,-71.076025,"(42.31841535, -71.07602496)"
1603,I182052602,111,Homicide,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",B2,324,Y,2018-07-04 21:50:00,2018,7,Wednesday,21,Part One,BROOKFORD ST,42.318415,-71.076025,"(42.31841535, -71.07602496)"
1648,I182052553,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,326,Y,2018-07-04 18:40:00,2018,7,Wednesday,18,Part One,FAYSTON ST,42.312243,-71.075499,"(42.31224327, -71.07549901)"


In [6]:
crime.describe()

,OFFENSE_CODE,YEAR,MONTH,HOUR,Lat,Long
count,969.000000,969.00000,969.00000,969.000000,936.000000,936.000000
mean,1087.329205,2016.47162,7.06192,13.519092,42.218083,-70.932742
std,1105.335769,0.95534,3.31137,8.114107,2.001098,3.237888
min,111.000000,2015.00000,1.00000,0.000000,-1.000000,-71.167152
25%,413.000000,2016.00000,5.00000,4.000000,42.294150,-71.091878
50%,413.000000,2017.00000,7.00000,16.000000,42.312517,-71.081363
75%,1841.000000,2017.00000,10.00000,20.000000,42.326112,-71.070215
max,3831.000000,2018.00000,12.00000,23.000000,42.389572,-1.000000


In [7]:
print(crime.count())

INCIDENT_NUMBER        969
OFFENSE_CODE           969
OFFENSE_CODE_GROUP     969
OFFENSE_DESCRIPTION    969
DISTRICT               967
REPORTING_AREA         969
SHOOTING               969
OCCURRED_ON_DATE       969
YEAR                   969
MONTH                  969
DAY_OF_WEEK            969
HOUR                   969
UCR_PART               964
STREET                 946
Lat                    936
Long                   936
Location               969
dtype: int64


In [8]:
crime.describe(include='all')

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
count,969,969.000000,969,969,967,969,969,969,969.00000,969.00000,969,969.000000,964,946,936.000000,936.000000,969
unique,599,NaN,27,55,12,259,1,598,NaN,NaN,7,NaN,4,322,NaN,NaN,484
top,I162067346,NaN,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,,Y,2016-08-20 00:05:00,NaN,NaN,Saturday,NaN,Part One,WASHINGTON ST,NaN,NaN,"(0.00000000, 0.00000000)"
freq,7,NaN,500,472,333,29,969,7,NaN,NaN,219,NaN,634,44,NaN,NaN,33
mean,NaN,1087.329205,NaN,NaN,NaN,NaN,NaN,NaN,2016.47162,7.06192,NaN,13.519092,NaN,NaN,42.218083,-70.932742,NaN
std,NaN,1105.335769,NaN,NaN,NaN,NaN,NaN,NaN,0.95534,3.31137,NaN,8.114107,NaN,NaN,2.001098,3.237888,NaN
min,NaN,111.000000,NaN,NaN,NaN,NaN,NaN,NaN,2015.00000,1.00000,NaN,0.000000,NaN,NaN,-1.000000,-71.167152,NaN
25%,NaN,413.000000,NaN,NaN,NaN,NaN,NaN,NaN,2016.00000,5.00000,NaN,4.000000,NaN,NaN,42.294150,-71.091878,NaN
50%,NaN,413.000000,NaN,NaN,NaN,NaN,NaN,NaN,2017.00000,7.00000,NaN,16.000000,NaN,NaN,42.312517,-71.081363,NaN
75%,NaN,1841.000000,NaN,NaN,NaN,NaN,NaN,NaN,2017.00000,10.00000,NaN,20.000000,NaN,NaN,42.326112,-71.070215,NaN


## Drop the rows where we are missing Latitude, Longitude information

We will drop the rows where we are missing lat, long information for the records (There are about 28k such records)

In [9]:
crime_subset = crime[(crime['Lat']!=-1) & (crime['Lat'].notnull())]

### crime_subset only contains records with latitudes and longitudes
- We will be using this subset of the data going forward

In [10]:
print(crime_subset.shape)
crime_subset.head()

(934, 17)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
759,I182053533,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,909,Y,2018-07-07 23:33:00,2018,7,Saturday,23,Part One,WHITTIER ST,42.333802,-71.088809,"(42.33380238, -71.08880867)"
869,I182053414,3001,Medical Assistance,DEATH INVESTIGATION,B2,316,Y,2018-07-07 15:06:00,2018,7,Saturday,15,Part Three,HOMESTEAD ST,42.312421,-71.091549,"(42.31242132, -71.09154902)"
1602,I182052602,3001,Medical Assistance,DEATH INVESTIGATION,B2,324,Y,2018-07-04 21:50:00,2018,7,Wednesday,21,Part Three,BROOKFORD ST,42.318415,-71.076025,"(42.31841535, -71.07602496)"
1603,I182052602,111,Homicide,"MURDER, NON-NEGLIGIENT MANSLAUGHTER",B2,324,Y,2018-07-04 21:50:00,2018,7,Wednesday,21,Part One,BROOKFORD ST,42.318415,-71.076025,"(42.31841535, -71.07602496)"
1648,I182052553,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,B2,326,Y,2018-07-04 18:40:00,2018,7,Wednesday,18,Part One,FAYSTON ST,42.312243,-71.075499,"(42.31224327, -71.07549901)"


## Load the library (uszipcodes) to map the latitude, longitudes to the zips
- To merge the dataset with the zip attributes, we would need to pull up the zips for each lat long present in the data file
- We create a unique values of latitude, longitude and pull up the zip for each set of lat, longs
- After pulling up the zips for each lat-long, we merge it back with the original dataset


In [11]:
from uszipcode import ZipcodeSearchEngine

ImportError: cannot import name 'ZipcodeSearchEngine' from 'uszipcode' (/Users/jaybennett/opt/anaconda3/lib/python3.8/site-packages/uszipcode/__init__.py)

In [ ]:
search = ZipcodeSearchEngine()

### Using the lat, long coordinates the library gives us the zip that it belongs to
Example for a random pair of lat, long gives us the zip for that lat long

In [ ]:
search.by_coordinate(42.362539, -71.165069, radius=2, returns=1)

### We take the unique values of latitude, longitudes to get the zip for each of them
- df_lat_long is the list of unique values for each of the 

In [ ]:
df_lat_long = crime_subset.loc[:,['Lat','Long']].drop_duplicates()

## The function below gives us a dictionary of zip and its demographic information through lat, long
- We apply this function to each row of the unique lat, long dataset
- We then append this new column of dicts of zip information to our unique lat long dataframe

In [ ]:
def get_demographics(lat_long):
    return search.by_coordinate(lat_long.Lat, lat_long.Long, radius=3, returns=1)

## CODE OPTIMIZATION:
- Initially the apply function was being run for the entire main dataset with 300k rows
- This was a bottleneck for us and initially it was taking us 15 minutes just to run the code below
- To optimize this step, we created a new dataframe of unique lat, longs and ran the apply function only for that
- This reduced the run time to 16 seconds

In [ ]:
#res = %timeit -o 
df_lat_long['zip_dict'] = df_lat_long.loc[:,['Lat','Long']].apply(get_demographics, axis=1)

In [ ]:
df_lat_long.head()

## Extract the zip code from the dictionary of information for each row

In [ ]:
df_lat_long = df_lat_long.assign(zip_code = df_lat_long.loc[:,['zip_dict']].apply(lambda x:[y[0].Zipcode for y in x]))

In [ ]:
df_lat_long.head()

### df_crime will be the main dataset which has the zips going forward

In [ ]:
df_crime = pd.merge(crime_subset, df_lat_long, how='left', on=['Lat', 'Long'], left_index=False, right_index=False)

### Verify whether the number of rows is the same in the merged dataset and the original one

In [ ]:
df_crime.shape[0] == crime_subset.shape[0]

In [ ]:
# df_crime.to_csv('crimes_with_locations.csv', index_label=False)

### Inspect that the zip column has been pulled into our original dataframe

In [ ]:
df_crime.head()

# Load the zip demographic information

In [ ]:
zip_demographics = pd.read_csv('zip_attributes.csv', dtype={'zip_code':np.object})
zip_demographics.head()

In [ ]:
crime_classes = pd.read_csv('crime_classification.csv')
crime_classes.head()

In [ ]:
df_crime_temp = df_crime.merge(zip_demographics, how='left', on='zip_code', left_index=False, right_index=False)
df_crime = df_crime_temp.merge(crime_classes, how='left', on='OFFENSE_DESCRIPTION', 
                                             left_index=False, right_index=False)

In [ ]:
df_crime.columns

In [ ]:
df_crime.shape

In [ ]:
grouped_zipcode = df_crime.groupby(['zip_code'])
demo_stats = grouped_zipcode.agg({'zip_code':'count','no_of_business':'max', 'median_age':'max', 'income_per_household':'max', 
                                  'business_first_quater_payroll':'max', 'avg_house_value':'max',
                                  'black_population':'max', 'white_population':'max'
                                 })
demo_stats.rename(columns={'zip_code':'num_of_crime_incidents'}, inplace=True)
demo_stats.head()

## Exploring the analysis between crime incidence and the demographic information
### Double-click the scatter-plots to zoom in

In [ ]:
y_vars=['num_of_crime_incidents']

sns.pairplot(demo_stats, x_vars=demo_stats.columns, y_vars=y_vars)

In [ ]:
sns.pairplot(demo_stats, kind="reg", vars=['num_of_crime_incidents', 'no_of_business', 'avg_house_value', 'black_population', 'white_population'])

## Exploring the crime variation with time information

In [ ]:
grouped_vars = df_crime.groupby(['DAY_OF_WEEK'])
grouped_stats = grouped_vars.agg({'zip_code':'count'})
grouped_stats.rename(columns={'zip_code':'num_of_crime_incidents'}, inplace=True)
grouped_stats= grouped_stats.reset_index()

sns.set(style="whitegrid")
day_of_week_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sns.barplot(x="DAY_OF_WEEK", y="num_of_crime_incidents", order=day_of_week_order, data=grouped_stats, color='lightblue')

In [ ]:
grouped_vars = df_crime.groupby(['HOUR'])
grouped_stats = grouped_vars.agg({'zip_code':'count'})
grouped_stats.rename(columns={'zip_code':'num_of_crime_incidents'}, inplace=True)
grouped_stats= grouped_stats.reset_index()

dims = (8, 6)
fig, ax = plt.subplots(figsize=dims)

hours_ranking = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,0,1,2,3,4,5]
sns.barplot(ax=ax, x="HOUR", y="num_of_crime_incidents", color='salmon', order=hours_ranking, data=grouped_stats)

## Exploring the crime variation with zips

In [ ]:
grouped_vars = df_crime.groupby(['zip_code'])
grouped_stats = grouped_vars.agg({'zip_code':'count'})
grouped_stats.rename(columns={'zip_code':'num_of_crime_incidents'}, inplace=True)
grouped_stats= grouped_stats.reset_index()

dims = (30, 5)
fig, ax = plt.subplots(figsize=dims)

sns.barplot(ax=ax, x="zip_code", y="num_of_crime_incidents", color='darkkhaki', data=grouped_stats)

# Regression Analysis:
- We see that crime varies over DAY_OF_WEEK, HOUR, and ZIP_CODE during exploratory analysis
- We will run a regression analysis using these 3 factors

### Results:
- Using these 3 variables as categorical variables in the equation, we see that we can explain __~82% of the variation in crime__
- This is __because the zip captures a lot of the demographic__ informatin within itself

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
grouped_vars = df_crime.groupby(['zip_code', 'HOUR', 'DAY_OF_WEEK'])
grouped_stats = grouped_vars.agg({'zip_code':'count'})
grouped_stats.rename(columns={'zip_code':'num_of_crime_incidents'}, inplace=True)
grouped_stats= grouped_stats.reset_index()

res_1 = smf.ols(formula='num_of_crime_incidents ~ C(DAY_OF_WEEK) + C(HOUR) + C(zip_code)', data=grouped_stats).fit()
print(res_1.summary())

In [ ]:
#this is done to merge the codes
crimes = df_crime

In [ ]:
#this is done to merge the codes
attributes = zip_demographics
attributes.population = attributes.population.replace(0,np.nan)
attributes = attributes.dropna(subset=['population'])

In [ ]:
crimes.head(10)

In [ ]:
zipcrime = crimes.groupby(by=['zip_code','HOUR','DAY_OF_WEEK']).agg({'INCIDENT_NUMBER':'count'}) #grouping the data by zip_code,hour and day of week
zipcrime = zipcrime.reset_index()
data = zipcrime.merge(attributes, how='left', left_on = zipcrime.zip_code, right_on=attributes.zip_code)
data = data.dropna(subset=['population']) #removing the rows which have zero population
data = data.drop(columns=[ 'zip_code_x', 'zip_code_y'])
print(len(data))
print(data.columns)

In [ ]:
data.head()

# Removing the variables based on economic significance and statistical significance. Running the linear regression to understand the correlations and probable causal relationship among attributes and CRIME_INCIDENCE

Following parameters were removed based on economic signinficance:
1. population: Since race wise population breakup is used, using this paramters will create multi-colineararity
2. delievery_total: the number of households which have a mail box. Since number of households are taken this variable is trivial.

Following parameters were statiscally insignificant:
3. business_first_quater_payroll
4. No_of_business
5. Median Age
6. Persons_per_household

In [ ]:
reg_data = data #creating suitable dataset for running the regression
reg_data = reg_data.drop(columns=['no_of_business','median_age','population','delivery_total','delivery_business','delivery_residential',
                                  'land_area','avg_house_value','business_first_quater_payroll','persons_per_household'])
reg_data.head()

In [ ]:
#running the linear regression
result = sm.ols(formula="INCIDENT_NUMBER ~ C(HOUR)+DAY_OF_WEEK+male_population+households_per_zip+income_per_household+asian_population+black_population+hawaiian_population+indian_population+other_population+white_population", data=data).fit()
print(result.summary())

# Clustering Algorithm

The statisically significant variables from the above regression model were taken to analyse the number of cluster present among the zip codes. Elbow graph showed that optimum number of clusters are 3. The clustering also supports the results from regression that the crime incidence is positively correlated with number of households, hawaiian population, indian population and negatively correlated with income per household, white population and black population

In [ ]:
zipcrime = crimes.groupby(by=['zip_code']).agg({'INCIDENT_NUMBER':'count'})
zipcrime = zipcrime.reset_index()
data = zipcrime.merge(attributes, how='left', left_on = zipcrime.zip_code, right_on=attributes.zip_code)
data = data.dropna(subset=['population'])
data = data.drop(columns=['zip_code_y','delivery_total',
                        'delivery_business','delivery_residential','land_area','avg_house_value',
                        'business_first_quater_payroll','persons_per_household','no_of_business','median_age'])
c_data = data.drop(columns=['zip_code_x','INCIDENT_NUMBER','population'])
data.head()

In [ ]:
sns.pairplot(c_data)

In [ ]:
rel_cols = list(c_data.columns)
nClusters=range(2,11)
sumDistances=[]
for n in nClusters:
    kmeans=KMeans(n_clusters=n).fit(c_data[rel_cols])
    sumDistances.append(kmeans.inertia_) #Proxy for SSE

In [ ]:
plt.plot(nClusters,sumDistances,'-')
plt.title('Variation of sum of errors(SSE) vs no of clusters')
plt.xlabel('No. of Clusters')
plt.ylabel('Sum Of Distances')
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=3).fit(c_data[rel_cols])
data['Cluster']=kmeans.labels_
g = sns.pairplot(data.iloc[:,1:],hue='Cluster')

In [ ]:
analysis = data.groupby(by=['Cluster']).agg({'zip_code_x':'count','INCIDENT_NUMBER':np.mean,'population':np.mean,
                                             'hawaiian_population':np.mean,'indian_population':np.mean,
                                             'income_per_household':np.mean,'households_per_zip':np.mean})
analysis.rename(columns={'zip_code_x':'num_of_zip_code','INCIDENT_NUMBER':'num_of_crime_incidence'}, inplace=True)
analysis

In [ ]:
print("List of zip code where the crime incidence is maximum hare the low income and highly populated areas\n", list(data.zip_code_x[data.Cluster==1]))

# Ploting the latitudes and longitudes on the map of boston using folium

In [ ]:
crime_by_lat_long = df_crime.groupby(['Lat','Long'])['zip_code'].count()
crime_by_lat_long = crime_by_lat_long.reset_index()

Below cell, plots Boston Heat Map of Crime Incidents, with co-ordinates(latitude,longitude) level granularity.Since the map plots ~18000 unique co-ordinates(latitudes,longitudes), size of the map will approx. be 6 MB and hence will not get displayed in Jupyter notebook. A .html file of one run is attached in zip file.Also once the below cell is executed, heatmap.html file gets saved in the current working directory.

In [ ]:
data = np.array(
    [
        crime_by_lat_long['Lat'],
        crime_by_lat_long['Long'],
        
    ]
).T

heatmap = folium.Map([42.36, -71.31], zoom_start=4) #latitude-longitude co-ordinates of Boston
heatmap.add_child(plugins.HeatMap([[row["Lat"], row["Long"]] for name, row in df_crime.iloc[:17997].iterrows()])) #Unique co-ordinates
plugins.MarkerCluster(data).add_to(heatmap)
heatmap.save("heatmap.html")


In [ ]:
end_time = time.time()
print("Total run time:",(end_time - start_time))

# Profiling the regression by artificially increasing the dataset size

The df_crime data is multiplied by 2,3,4 and so till 9 in every iteration

In [ ]:
initial_dataset = df_crime

data_size_multiplier = [1, 2, 3, 4, 5, 6, 7, 8]
time_taken = []

for num, size in enumerate(data_size_multiplier):
    dataset = initial_dataset.append([initial_dataset]*size,ignore_index=True)
    initial_dataset = df_crime
    print("========================================================")
    %memit
    print("DATASET SIZE: {:,}".format(dataset.shape[0]))
    t_1 = time.time()
    grouped_vars = dataset.groupby(['zip_code', 'HOUR', 'DAY_OF_WEEK'])    
    grouped_stats = grouped_vars.agg({'zip_code':'count'})
    grouped_stats.rename(columns={'zip_code':'num_of_crime_incidents'}, inplace=True)
    grouped_stats= grouped_stats.reset_index()
    
    res_1 = smf.ols(formula='num_of_crime_incidents ~ C(DAY_OF_WEEK) + C(HOUR) + C(zip_code)', data=grouped_stats).fit()
    t_2 = time.time()
    time_taken.append(t_2-t_1)
    print("TIME TAKEN: {}".format(t_2 - t_1), end='\n\n')

data_size_labels = [x+1 for x in data_size_multiplier]

### Time taken to run increases linearly with increase in the original datasize (original size = ~300k)

In [ ]:
fig, ax = plt.subplots()

plt.title('Time to run the regression vs Datasize multiplier', fontsize=16)
plt.ylabel('Time in seconds', fontsize=15)
plt.xlabel('X TIMES Original Datasize', fontsize=15)
ax.plot(data_size_labels, time_taken)

# Profiling of clustering algorithm and ploting the pairplot with increasing zip codes

The dataset is multiplied by 4 in every iteration to analyse the time taken to perform clustering algorithm and plotting the pairplots between given attributes. We see an increase in time with an almost linear relationship when the number of zip codes approaches 41,000 which is equal to the total number of zip codes in US.

Hence, if this algorithm is run for the complete US data over given attributes, it will take just 34 seconds run time.

In [ ]:
initial_dataset = c_data

data_size_multiplier = [3,3,3,3,3]
time_taken = []

for num, size in enumerate(data_size_multiplier):
    print("========================================================")
    %memit
    dataset = initial_dataset.append([initial_dataset]*size,ignore_index=True)
    initial_dataset = dataset
    print("DATASET SIZE: {:,}".format(dataset.shape[0]))
    t_1 = time.time()
    kmeans=KMeans(n_clusters=3).fit(dataset[rel_cols])
    dataset['Cluster'] = kmeans.labels_
    g = sns.pairplot(dataset,hue='Cluster')
    t_2 = time.time()
    time_taken.append(t_2-t_1)
    print("TIME TAKEN: {}".format(t_2 - t_1), end='\n\n')

data_size_labels = [4,16,64,256,1024]

In [ ]:
fig, ax = plt.subplots()

plt.title('Time to run the clustering algorithm vs Datasize multiplier', fontsize=16)
plt.ylabel('Time in seconds', fontsize=15)
plt.xlabel('X TIMES Original Datasize', fontsize=15)
ax.plot(data_size_labels, time_taken)